In [1]:
#the idea is to predict the outcome of the match at the 15 minute mark, training and testing multiple models and parameter choices

import pandas as pd
import numpy as np
import ast
from patsy import dmatrices
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

In [2]:
#Loading the complete data and sampling the top few rows to print format
complete_df = pd.read_csv("leagueoflegends/_LeagueofLegends.csv")
complete_df[['bResult','goldblue','goldred','bKills','rKills','bTowers','rTowers','bInhibs','rInhibs','bDragons','rDragons','bHeralds','rHeralds','bBarons','rBarons']].head()

,bResult,goldblue,goldred,bKills,rKills,bTowers,rTowers,bInhibs,rInhibs,bDragons,rDragons,bHeralds,rHeralds,bBarons,rBarons
0,1,"[2415, 2415, 2711, 3887, 5068, 6171, 7412, 866...","[2415, 2415, 2725, 3952, 5336, 6602, 7900, 945...","[[10.82, 'C9 Hai', 'TSM Bjergsen'], [16.286, '...","[[16.529, 'TSM Lustboy', 'C9 Balls', 'C9 Meteo...","[23.038, 15.217, 15.014, 33.583, 39.269, 39.37...","[20.681, 39.23, 15.288, 17.237]","[36.686, 29.274]",[],[37.267],"[24.641, 17.14, 30.934]",[],[],[],[29.954]
1,0,"[2415, 2415, 2705, 4108, 5511, 6797, 7637, 895...","[2415, 2415, 2731, 4126, 5364, 6560, 7789, 893...","[[11.104, 'DIG Shiphtur', 'CST Jesiz', 'CST Im...","[[12.387, 'CST Jesiz', 'DIG Gamsu', 'DIG Shiph...","[33.018, 34.766, 23.239, 20.058, 15.306, 25.56...","[19.257, 23.409, 37.008, 29.993, 15.206, 36.93...",[],[36.813],"[19.119, 26.177, 32.545]",[12.264],[],[],[29.255],[]
2,1,"[2415, 2415, 2726, 3794, 4933, 6236, 8109, 965...","[2415, 2415, 2716, 3854, 4899, 6199, 7520, 859...","[[5.255, 'GV Keane', 'WFX Pobelter', 'WFX Shor...","[[8.449, 'WFX Altec', 'GV Cop', 'GV BunnyFuFuu...","[18.294, 36.946, 32.833, 25.481, 29.505, 38.44...","[30.493, 24.62]","[33.289, 37.38, 39.322, 37.511]",[],"[18.255, 24.577, 30.87, 37.867]",[],[],[],[35.144],[]
3,0,"[2415, 2415, 2705, 3847, 5398, 6473, 7720, 930...","[2415, 2415, 2720, 3822, 5170, 6479, 7963, 913...","[[8.274, 'TL Quas', 'TIP Rhux', 'TIP Apollo'],...","[[7.768, 'TIP Rush', 'TL IWDominate', 'TL Feni...","[22.879, 18.541, 19.941, 38.77]","[39.924, 27.034, 31.665, 36.384, 29.969, 19.12...",[],[35.867],[],"[18.515, 10.153, 26.274, 32.796]",[],[],[37.513],[]
4,1,"[2415, 2415, 2710, 3950, 5404, 6666, 7887, 913...","[2375, 2375, 2666, 3986, 5291, 6508, 8008, 932...","[[11.438, 'T8 Dodo8', 'CLG Doublelift', 'CLG X...","[[11.988, 'CLG Doublelift', 'T8 Porpoise8', 'T...","[33.925, 26.963, 22.594, 34.515, 34.213, 14.97...","[11.644, 12.438, 15.252, 31.49]",[34.069],[],"[30.307, 14.589]",[21.901],[],[],[32.556],[]


In [3]:
#Checking if a given list is empty or not, can handle nested lists
def empty(seq):
	try:
		return all(map(empty, seq))
	except TypeError:
		return False

In [4]:
#Function to convert string containing values to list and add integers
def to_list(series):
    return_series = series.str.strip('[').str.strip(']').str.split(',')
    return return_series.map(lambda x: list(map(lambda y: int(y), x)))

In [5]:
#Converting elements from list of values to count of values before 15min and storing in data frame
def set_15min_count(index,var,name):
	if(empty(var)==False):
		count  = len([item for item in var if item<15])
	else: 
		count = 0
	complete_df.set_value(index,name,count)

In [6]:
#input variables that we require all all in the form of string of data ie. "[[0,2,4],[3,4,5]]" converting to list of list and then computing count of occurances before 15 minutes
def sanitise_count():    
	complete_df.goldblue = to_list(complete_df.goldblue);
	complete_df.goldred = to_list(complete_df.goldred);
	
	for index,game in complete_df.iterrows():
		complete_df.set_value(index,'goldblue',game.goldblue[15])
		complete_df.set_value(index,'goldred',game.goldred[15])
		var_bk = ast.literal_eval(game.bKills)
		var_rk= ast.literal_eval(game.rKills)
		set_15min_count(index,ast.literal_eval(game.bTowers),'bTowers')
		set_15min_count(index,ast.literal_eval(game.rTowers),'rTowers')
		set_15min_count(index,ast.literal_eval(game.rInhibs),'rInhibs')
		set_15min_count(index,ast.literal_eval(game.bInhibs),'bInhibs')
		set_15min_count(index,ast.literal_eval(game.bDragons),'bDragons')
		set_15min_count(index,ast.literal_eval(game.rDragons),'rDragons')
		set_15min_count(index,ast.literal_eval(game.bHeralds),'bHeralds')
		set_15min_count(index,ast.literal_eval(game.rHeralds),'rHeralds')
		set_15min_count(index,ast.literal_eval(game.bBarons),'bBarons')
		set_15min_count(index,ast.literal_eval(game.rBarons),'rBarons')
		if(empty(var_bk)==False):
			count  = len([item[0] for item in var_bk if item[0]<15])
		else: 
			count = 0
		complete_df.set_value(index,'bKills',count)
		if(empty(var_rk)==False):
			count  = len([item[0] for item in var_rk if item[0]<15])
		else: 
			count = 0
		complete_df.set_value(index,'rKills',count)

In [7]:
#sanitising input and sampling data
sanitise_count()
complete_df[['bResult','goldblue','goldred','bKills','rKills','bTowers','rTowers','bInhibs','rInhibs','bDragons','rDragons','bHeralds','rHeralds','bBarons','rBarons']].head()

,bResult,goldblue,goldred,bKills,rKills,bTowers,rTowers,bInhibs,rInhibs,bDragons,rDragons,bHeralds,rHeralds,bBarons,rBarons
0,1,19952,20563,1,0,0,0,0,0,0,0,0,0,0,0
1,0,21528,20227,5,3,0,0,0,0,0,1,0,0,0,0
2,1,22738,19664,8,3,0,0,0,0,0,0,0,0,0,0
3,0,20408,20416,2,4,0,0,0,0,0,1,0,0,0,0
4,1,21404,21227,1,1,1,2,0,0,1,0,0,0,0,0


In [8]:
#the idea is to predict the outcome of the match at the 15 minute mark based on all ten parameters chosen
#function to generate the appropriate dataframes for the scikit functions, dividing data into test and traing data and measuring the correctness of the test data
def learn(type):
	y, X = dmatrices('bResult ~ goldblue + goldred + bKills + rKills + bTowers + rTowers + bInhibs + rInhibs + bDragons 	+ rDragons + bHeralds + rHeralds + bBarons + rBarons',complete_df, return_type="dataframe")
	y = np.ravel(y)
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
	model2 = type()
	model2.fit(X_train, y_train)
	predicted = model2.predict(X_test)
	probs = model2.predict_proba(X_test)
	print('Accuracy',' : ',metrics.accuracy_score(y_test, predicted))
	print('Roc Score',' : ',metrics.roc_auc_score(y_test, probs[:, 1]))

In [9]:
#learn(KNeighborsClassifier)
print('Logistic Regression :')
learn(LogisticRegression)

Logistic Regression :
Accuracy  :  0.710562414266
Roc Score  :  0.779484301509


In [10]:
#here we are now going to divide the parameters which include data for red team and blue team into two sets of parameters
#this may increase the size of dataset and give better result or as the data is no longer relative ie. both teams are not being compared in a single game ,it may drop
def divide_redblue():	
	var = complete_df[['rResult','goldred','rKills','rTowers','rInhibs','rDragons','rHeralds','rBarons']]
	var.index = range(3645,7290)
	collapse_df['result'] = pd.concat([complete_df['bResult'], var['rResult']])
	collapse_df['gold'] = pd.concat([complete_df['goldblue'], var['goldred']])
	collapse_df['kills'] = pd.concat([complete_df['bKills'], var['rKills']])
	collapse_df['tower'] = pd.concat([complete_df['bTowers'], var['rTowers']])
	collapse_df['inhibs'] = pd.concat([complete_df['bInhibs'], var['rInhibs']])
	collapse_df['dragons'] = pd.concat([complete_df['bDragons'], var['rDragons']])
	collapse_df['herald'] = pd.concat([complete_df['bHeralds'], var['rHeralds']])
	collapse_df['baron'] = pd.concat([complete_df['bBarons'], var['rBarons']])

In [11]:
#create a new dataframe with the divided values we need
columns = ['result','gold','kills','tower','inhibs','dragons','herald','baron']
collapse_df = pd.DataFrame(index=range(0,7290),columns=columns);
divide_redblue()
collapse_df.head()

,result,gold,kills,tower,inhibs,dragons,herald,baron
0,1,19952,1,0,0,0,0,0
1,0,21528,5,0,0,0,0,0
2,1,22738,8,0,0,0,0,0
3,0,20408,2,0,0,0,0,0
4,1,21404,1,1,0,1,0,0


In [12]:
#function to generate the appropriate dataframes for the scikit functions, dividing data into test and traing data and measuring the correctness of the test data
def learn(type):
	y, X = dmatrices('result ~ gold + kills + tower + inhibs + dragons + herald + baron',collapse_df, return_type="dataframe")
	y = np.ravel(y)
	X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
	model2 = type()
	model2.fit(X_train, y_train)
	predicted = model2.predict(X_test)
	probs = model2.predict_proba(X_test)
	print('Accuracy',' : ',metrics.accuracy_score(y_test, predicted))
	print('Roc Score',' : ',metrics.roc_auc_score(y_test, probs[:, 1]))

In [13]:
#learn(KNeighborsClassifier)
print('Logistic Regression :')
learn(LogisticRegression)

#the accuracy of this model is ~5% worst than regression with all ten parameters

Logistic Regression :
Accuracy  :  0.644718792867
Roc Score  :  0.693137629675
